# Imports and utilities

In [19]:
%reload_ext autoreload

In [ ]:
import json
import pandas as pd
from shared.snowflake.client import SnowflakeClient

sf_client = SnowflakeClient()
session = sf_client.get_snowpark_session()

# Main code

## Design code and execute it

In [57]:
from typing import List, Any, Dict
from snowflake.snowpark import Session


def parse_procedure_result(query_result, proc_name) -> Any:
    """
    Parse a procedure result parsed with query result to be usable.
    Args:
        query_result: query result parsed
        proc_name: procedure name

    Returns:
        output: Any
    """
    value = query_result[0][proc_name]
    output = json.loads(value)
    return output


def parse_query_result(query_result) -> List[Dict[str, float]]:
    """
    Collect query result and return as dict list
    Args:
        query_result : result of a query call (session.sql(query))

    Returns:
        List[Dict[str, float]]: formatted output
    """
    collected_result = query_result.collect()
    return [row.as_dict() for row in collected_result]


def format_output(input: Any) -> str:
    """
    Dumps output in json format to be usable.
    Args:
        input: Any type of data
    Returns:
        str: json result of the formatted output
    """
    # Convertir les Decimal en float pour la sérialisation JSON
    if (
        isinstance(input, list)
        and len(input) > 0
        and isinstance(input[0], dict)
    ):
        from decimal import Decimal

        for item in input:
            for key, value in item.items():
                if isinstance(value, Decimal):
                    item[key] = float(value)

    # Retourner en JSON
    return json.dumps(input, indent=2)

In [ ]:
def procedure_transform_service(
    session: Session,
    recipe: dict,
    ingredients_to_remove: list[str],
    constraints: dict,
) -> str:
    """
    Transform service

    Args:
    recipe: Dictionary containing the recipe details.
    ingredients_to_remove: List of ingredients to be removed.
    constraints: Dictionary containing transformation constraints.

    Returns:
    JSON string containing the query results.
    """

    # Query to fetch ingredient details
    query = """
    SELECT 
        ci."DESCRIP",
        ci."PROTEIN_G", 
        ci."SATURATED_FATS_G",
        ci."FAT_G",
        ci."CARB_G", 
        ci."SODIUM_MG",
        ci."FIBER_G",
        ci."SUGAR_G",
        ci."ENERGY_KCAL",
        ci."NDB_NO",
    FROM NUTRIRAG_PROJECT.RAW.CLEANED_INGREDIENTS ci
    LIMIT 5;
    """
    result_query = session.sql(query)  # Run sql query
    data_list = parse_query_result(result_query)  # Parsed query result

    output = format_output(data_list)  # Dumps data in json format
    return output

## Get code 
Reads the code stored in the file_path.

In [72]:
def read_code(file_path):
    """Lit le code Python qui contient la logique (sp_handler, imports, etc.)"""
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

In [74]:
# file_path = "YOUR_FILE_PATH"
file_path = "/Users/sushiatomique/Documents/M2/NutriRAG/backend/app/udf/snowflake_procedure.py"
code = read_code(file_path)

This procedure run a sql query and return the result in an usable format.

In [75]:
print(code)

import json
from decimal import Decimal
from typing import List, Any, Dict
from snowflake.snowpark import Session


def parse_procedure_result(query_result, proc_name)->Any:
    """
    Parse a procedure result parsed with query result to be usable.
    Args:
        query_result: query result parsed
        proc_name: procedure name

    Returns:
        output: Any 
    """
    value = query_result[0][proc_name]
    output = json.loads(value)
    return output

def parse_query_result(query_result)->List[Dict[str, float]]:
    """
    Collect query result and return as dict list
    Args:
        query_result : result of a query call (session.sql(query))

    Returns:
        List[Dict[str, float]]: formatted output
    """
    collected_result = query_result.collect()
    return [row.as_dict() for row in collected_result]

def format_output(input: Any)->str:
    """
    Dumps output in json format to be usable.
    Args:
        input: Any type of data 
    Returns:
        str: json

## Register code as Procedure Snowflake

In [76]:
# 3. Déploiement de la Procédure Stockée
proc_name = "TRANSFORM_TEMPLATE"
main_func_name = "procedure_template"
args = ["ARG1", "ARG2"]

In [77]:
query = f"""
    CREATE OR REPLACE PROCEDURE {proc_name}({" STRING, ".join(args) + " STRING"})
    RETURNS STRING
    LANGUAGE PYTHON
    RUNTIME_VERSION = '3.10'
    PACKAGES = ('snowflake-snowpark-python', 'filelock')
    EXTERNAL_ACCESS_INTEGRATIONS = (TRAINING_INTERNET_ACCESS)
    HANDLER = '{main_func_name}'
    EXECUTE AS OWNER
    AS
    $${code}$$
"""

In [78]:
result = session.sql(query).collect()

In [79]:
result

[Row(status='Function TRANSFORM_TEMPLATE successfully created.')]

## Call example

In [80]:
query = f"""
CALL {proc_name}(
    'yes',
    'yes'
)
"""

result = session.sql(query)

In [83]:
parsed_output = parse_procedure_result(parse_query_result(result), proc_name)

In [85]:
df_output = pd.DataFrame(parsed_output)

In [86]:
df_output

,DESCRIP,PROTEIN_G,SATURATED_FATS_G,FAT_G,CARB_G,SODIUM_MG,FIBER_G,SUGAR_G,ENERGY_KCAL,NDB_NO
0,butter with salt,0.85,51.368,81.11,0.06,643.0,0.0,0.06,717.0,1001
1,butter whipped w salt,0.49,45.390,78.30,2.87,583.0,0.0,0.06,718.0,1002
2,butter oil anhydrous,0.28,61.924,99.48,0.00,2.0,0.0,0.00,876.0,1003
3,cheese blue,21.40,18.669,28.74,2.34,1146.0,0.0,0.50,353.0,1004
4,cheese brick,23.24,18.764,29.68,2.79,560.0,0.0,0.51,371.0,1005


# Close connection

In [87]:
sf_client.close()